
# The Truth about Objects

### Naomi Ceder
**This notebook -  https://github.com/nceder/talks**
![Image](qr-code.png)


## In Python "everything is an object" 
### Everything?
* A constant?
* An instance of a class?
* A function?
* A module/package?


### Simple types

In [ ]:
print(f"-1 is instance of object - {isinstance(-1, object)}\n"
      f"(-1).__class__ - {(-1).__class__}")

### What about functions?

In [ ]:
def test():
    print("test")

print(f"test is instance of object - {isinstance(test, object)}\n" 
      f"test.__class__ - {test.__class__}")


### Modules?

In [ ]:
import math
print(f"math is instance of object - {isinstance(math, object)}\n" 
      f"math.__class__ - {math.__class__}")

### Other stuff?

In [ ]:
print(f"None is instance of object - {isinstance(None, object)}\n" 
      f"None.__class__ - {None.__class__}")

In [ ]:
print(f"... is instance of object - {isinstance(..., object)}\n" 
      f"....__class__ - {....__class__}")

In [ ]:
# +-*/%//,.[]: 
isinstance(-, object)

In [ ]:
# in not if elif else for while def class...  
isinstance(in, object)

## Looks like *almost* everything else is an object...
### and an instance of a class... 

 so... 

**what about classes?** 

### Sample class

In [ ]:
class Duck:
    pass

donald = Duck()
print(f"donald is instance of object - {isinstance(donald, object)}\n" 
      f"donald.__class__ - {donald.__class__}")

In [ ]:
print(f"Duck is instance of object - {isinstance(Duck, object)}\n" 
      f"Duck.__class__ (i.e, type(Duck)) - {Duck.__class__}")

### `object` and `type`

In [ ]:
print(f"type is instance of object - {isinstance(type, object)}\n" 
      f"type.__class__ - {type.__class__}")


In [ ]:
print(f"object.__class__ - {object.__class__}")

In [ ]:
# we saw above that type is an instance of object... 

print(f"object is instance of type - {isinstance(object, type)}")

In [ ]:
object == type

In [ ]:
print(f"object is subclass of type - {issubclass(object, type)}")

In [ ]:
print(f"type is subclass of object - {issubclass(type, object)}")

### So...in Python, 

* **everything (almost) is an instance of `object`**, including classes (i.e. `type`) and `object` itself.
* **but... `object` is an instance of `type`.** (and class == type) 



![Image](mc_escher_drawing_hands_1948.jpg)

## Classes

**But how do classes and instances work?**

### What's a class?

* an object of type `type`
* created with... 
   * `class` keyword
   * class name
* Instance methods
   * `def` keyword and method name
   * parameter reference to instance (self)
* Instance data
   * data attributes of an instance
   * instance referred to as “self” (we could use any legal identifier)

In [ ]:
class Duck:
    def __init__(self, name="a duck"):
        self.name = name
    
    def hello(self):
        print(f"Hello, I'm a {self.__class__.__name__}" 
              f" named {self.name}")

In [ ]:
donald = Duck("Donald")
donald.hello()

In [ ]:
Duck.__dict__

In [ ]:
print(donald)
donald.__dict__

### The other way to make a class

* if we can make an `int` with `int()`, can we make a class (`type`) with `type()`?


In [ ]:
def init_funct(self, name):
    self.name = name
def str_funct(self):
    return f"I am a {self.__class__.__name__} named {self.name}"

namespace = {'__init__': init_funct, 
             '__str__':str_funct }

namespace

In [ ]:
Scratch = type('Scratch', (), namespace)

print(Scratch)
Scratch.__dict__

In [ ]:
my_scratch = Scratch('my_scratch')
print(my_scratch)
my_scratch.__dict__

### Classes can be created using <br><br>`type(<classname>, <bases>, <dict>)`

### More on methods

* an instance method is “bound”, i.e. it is called on an instance
* `a_class.method(an_instance, param) == an_instance.method(param)`

In [ ]:
class Duck:
    def __init__(self, name="a duck"):
        self.name = name
    def hello(self):
        return (f"I'm a {self.__class__.__name__}"
                f" named {self.name}")
    def goodbye(self):
        return (f"Goodbye (goodbye method) "
                f"from {self.name}")
donald = Duck("Donald")
print(donald.hello())
print(donald.goodbye())

In [ ]:
print(Duck.hello(donald))
print(Duck.goodbye(donald))

In [ ]:
# remember 'my_scratch' from above?
print(Duck.hello(my_scratch))
print(Duck.goodbye(my_scratch))

### What about changing Duck?



In [ ]:
class Duck:
    def __init__(self, name="a duck"):
        self.name = name
    def hello(self):
        return (f"I'm a {self.__class__.__name__}"
                f" named {self.name}")
    def goodbye(self):
        return (f"Goodbye (goodbye method) "
                f"from {self.name}")
    
def farewell(thing):
    return (f"Farewell (farewell function) from "
            f"{thing.name}")

In [ ]:
donald = Duck("Donald")

print(donald.goodbye())

print(farewell(donald))

In [ ]:
old_goodbye = Duck.goodbye

Duck.goodbye = farewell

daisy = Duck("Daisy")

print(daisy.goodbye())

In [ ]:
print(donald.goodbye())

In [ ]:
del Duck.goodbye

daisy.goodbye()

In [ ]:
Duck.goodbye = old_goodbye

daisy.goodbye()

### Bound methods 
* are in the class, not the instance
* the class is an object that can be changed

#### Changing the class on the fly is "monkey patching" 
* It can make things complicated, if not impossible, to debug
* so **don't try this at home!**<br>
* *(or do, I won't stop you, but be warned... here be dragons...)*


## Metaclasses

* In Python everything is an object, even classes (types)
* Suppose we inherited from `type` and changed a few things?
* We could define classes that behave differently
* e.g. classes that automatically register themselves when instantiated


### Metaclass that makes only singleton classes

We could create a new *type* of class that **only** creates singleton classes... 

* inherit from `type`
* overload `__call__` method to 
   * check for existing instance 
   * before creating/saving new instance
   * return saved instance

In [ ]:
class ClassSngl(type):
    def __call__(cls, *args, **kwds):
        if not hasattr(cls, "instance"):
            print(f"Calling {cls.__name__}() "
                  "to create instance")
            cls.instance = type.__call__(cls, 
                        *args, **kwds)
        print("returning instance")
        return cls.instance
    
type(ClassSngl)

In [ ]:
class Sngltn(metaclass=ClassSngl):
    def __init__(self, *args, **kwds):
        print(f"Initializing new instance of {self.__class__.__name__} ")
        
Sngltn.__class__

In [ ]:
single_1 = Sngltn()

In [ ]:
single_2 = Sngltn()

In [ ]:
print(id(single_1))
print(id(single_2))

In [ ]:
class AltSngltn(metaclass=ClassSngl):
    def __init__(self, *args, **kwds):
        print(f"Initializing new instance of {self.__class__.__name__} ")

In [ ]:
alt_single_1 = AltSngltn()
alt_single_2 = AltSngltn()
print(f"{id(alt_single_1)=}\n{id(alt_single_1)=}")

### Metaclasses can change how classes behave

Again, **not** recomended, other than for experimentation...

Metaclasses in production - if you don't know you really need them, you probably don't need them. 

## Conclusions

* Everything (almost) is an object
* Classes are objects, too
* Classes can be altered dynamically
* Metaclasses can change how classes behave

### Questions?


**This notebook - https://github.com/nceder/talks**

![Image](qr-code.png)<br><br><br><br><br><br><br>

### Bonus: A metaclass that will create only one class

**A type singleton?**

In [ ]:
class SnglClass(type):
    def __new__(meta, name, bases, dct):
        print(f"Calling {meta.__new__}() to create class")
        if not hasattr(meta, "class_"):
            print("creaing new class")
            meta.class_ = type.__new__(meta, name, bases, dct)
        print("returning class")
        return meta.class_

In [ ]:
class test(metaclass=SnglClass):
    pass

class test2(metaclass=SnglClass):
    pass

In [ ]:
test2

## Abstract and outline

"Everything in Python is an object." This is a profound truth about Python, but what does it mean? Is literally EVERYTHING an object? And what is an object anyway? Are objects the same as instances of a class? How do classes and types really work in Python? And what do metaclasses have to do with anything?

In fact, the answers to these questions are probably not what you think they are - Python's approach to objects is different from most other languages in sometimes surprising ways. 

This talk will use simple live coded examples to explore how objects work in Python and clear up several common misconceptions and misunderstandings about how objects and instances, classes and types, and metaclasses all work together. 

Be warned - you are likely to be surprised when you learn the truth about objects in Python.

1. Introduction - 3 min
2. Is everything really  an object? - 5 min
3. class = type - 5 min
    1. What is the type of a class?
    2. Constructing a class using type()
5. What is an instance? - 5 min
     1. How is an instance connected to a class?
     2. Bound methods and monkeypatching
6. Type vs. metaclass - 5 min
     1. Changing how a class is made/behaves
7. Conclusion/Questions - 5 min